# Dataset 확인

## 기본 영상 길이 넘는지 확인

In [2]:
import cv2
import os
import pandas as pd

person_0 = []
person_1 = []
person_2 = []
person_3 = []
person_4 = []

data_dir = 'korea_data'
data_per = os.listdir(data_dir)
for person in data_per:
    words_dir= f'{data_dir}/{person}'
    Id = os.listdir(words_dir)
    for id in Id:
        if id == '.DS_Store':
            continue
        id_dir = f'{data_dir}/{person}/{id}'
        num = os.listdir(id_dir)
        if len(num) == 3:
            num = os.listdir(id_dir)
            for n  in num:
                words_dir_2 = f'{data_dir}/{person}/{id}/{n}/crop_lip'
                frames = os.listdir(words_dir_2)
                if frames == '.DS_Store':
                    continue
                if len(frames) < 5:
                    if person == '00':
                        person_0.append((id, n))
                    elif person == '01':
                        person_1.append((id, n))
                    elif person == '02':
                        person_2.append((id, n))
                    elif person == '03':
                        person_3.append((id, n))
                    else:
                        person_4.append((id, n))
print(len(person_0),len(person_1),len(person_2),len(person_3),len(person_4))

0 0 0 0 0


## 데이터셋 전처리(crop)
- 입술 주변을 Detect 후 100x50 pixel crop

In [3]:
def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((20, 2), dtype=dtype)
    # for only lip landmarks
    n = 0
    for i in range(48, shape.num_parts):
        coords[n] = (shape.part(i).x, shape.part(i).y)
        n += 1

    # return the list of (x, y)-coordinates
    return coords

In [5]:
## Dataset crop Demo
from keras.preprocessing import image
import cv2
import os
import pandas as pd
import dlib
import numpy as np

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat") # predict 해주는애            

data_dir = 'korea_data'
data_per = os.listdir(data_dir)
for person in data_per:
    for sentence in os.listdir(f'{data_dir}/{person}' ):

        if sentence == 'csv' or sentence == 'video' or sentence == '.DS_Store':
            continue
        for nums in os.listdir(f'{data_dir}/{person}/{sentence}'):
            if nums == '.DS_Store':
                continue
            n = 0
            for jpg in os.listdir(f'{data_dir}/{person}/{sentence}/{nums}/image'):
                # 3초 안으로
                if int(jpg[4]) > 4:
                    break
                faces = f'{data_dir}/{person}/{sentence}/{nums}/image/{jpg}'
                src = cv2.imread(faces, cv2.IMREAD_COLOR)
                img_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
                face_detector = detector(img_gray, 1)
                for face in face_detector:
                    landmarks = predictor(src, face)
                    landmarks = shape_to_np(landmarks)
                    # select center of mouth
                    x_list = [x[0] for x in landmarks]
                    y_list = [y[1] for y in landmarks]
                    x = sum(x_list)//20
                    y = sum(y_list)//20
                dst = src[y-50:y+50, x-100:x+100].copy()
                # 확인용
                #cv2.imshow("src", src)
                #cv2.imshow("dst", dst)
                #cv2.waitKey()
                #cv2.destroyAllWindows()
                img_tensor = image.img_to_array(dst) #tensor로 type 변경
                img_tensor = img_tensor.flatten() # flatten: 12288 차원
                img_tensor /= 255. # scaling 
                img_tensor = np.expand_dims(img_tensor, axis = 0) # (1, 60000) 차원

[[0.4117647  0.4        0.43529412 ... 0.3137255  0.24313726 0.21568628]]
60000
[[0.4        0.3882353  0.42352942 ... 0.3137255  0.24313726 0.23921569]]
60000
[[0.40784314 0.41960785 0.4509804  ... 0.26666668 0.16862746 0.12941177]]
60000
[[0.39607844 0.40784314 0.4392157  ... 0.2509804  0.16078432 0.12941177]]
60000
[[0.39215687 0.39607844 0.43529412 ... 0.30588236 0.21568628 0.18431373]]
60000
[[0.4392157  0.4509804  0.48235294 ... 0.36078432 0.2627451  0.2       ]]
60000
[[0.43137255 0.4392157  0.47843137 ... 0.32156864 0.20392157 0.16078432]]
60000
[[0.44313726 0.44705883 0.4862745  ... 0.27450982 0.19607843 0.15294118]]
60000
[[0.43529412 0.45490196 0.49019608 ... 0.27058825 0.16470589 0.1254902 ]]
60000
[[0.4392157  0.46666667 0.5019608  ... 0.25882354 0.15294118 0.11372549]]
60000
[[0.44313726 0.45882353 0.5019608  ... 0.30980393 0.21176471 0.17254902]]
60000
[[0.43529412 0.4509804  0.49411765 ... 0.35686275 0.25490198 0.22745098]]
60000
[[0.45882353 0.46666667 0.5058824  ... 0

KeyboardInterrupt: 